# 에이전트 평가(Evaluating Agents)

이메일 어시스턴트는 라우터를 사용하여 이메일을 분류한 후 응답 생성을 위해 에이전트에게 전달합니다. 실제 운영 환경에서 제대로 작동할지 어떻게 확신할 수 있을까요? 그래서 테스트가 중요한 이유입니다: 응답 품질, 토큰 사용량, 지연 시간, 분류 정확도 같은 정량적 지표를 통해 에이전트 아키텍처에 대한 결정을 안내합니다. 

[LangSmith](https://docs.smith.langchain.com/)는 에이전트 테스트를 위한 두 가지 주요 방법을 제공합니다. 

![overview-img](assets/overview_eval.png)

## Resources

- Notebook Reference: [evaluation.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/evaluation.ipynb)
- For running evaluations: [test_tools.py](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/test_tools.py), [/tests/](https://github.com/langchain-ai/agents-from-scratch/tree/main/tests)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)

## 환경 설정

In [ ]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)

## 평가 실행 방법

#### Pytest / Vitest

[Pytest](https://docs.pytest.org/en/stable/)와 Vitest는 Python 및 JavaScript 생태계 내에서 테스트를 작성하는 강력한 도구입니다. LangSmith는 이러한 프레임워크와 통합되어 결과를 LangSmith에 기록하는 테스트를 작성하고 실행할 수 있도록 합니다. 이 노트북에서는 Pytest를 사용합니다.
* Pytest는 해당 프레임워크에 이미 익숙한 개발자가 시작하기에 좋은 방법입니다.
* Pytest는 각 에이전트 테스트 케이스에 일반화하기 어려운 특정 검사와 성공 기준이 필요한 더 복잡한 평가에 적합합니다.

#### LangSmith 데이터셋

LangSmith에서 데이터셋을 생성하고 LangSmith evaluate API를 사용하여 해당 데이터셋에 대해 어시스턴트를 실행할 수 있습니다.
* LangSmith 데이터셋은 팀이 협업으로 test suite를 구축하는 데 적합합니다.
* 프로덕션 추적 기록, 어노테이션 대기열, 합성 데이터 생성 등을 활용하여 지속적으로 확장되는 골든 데이터셋에 예시를 추가할 수 있습니다.
* LangSmith 데이터셋은 데이터셋 내 모든 테스트 케이스에 적용 가능한 평가기(예: 유사도, 정확 일치 정확도 등)를 정의할 수 있을 때 특히 유용합니다.

테스트 케이스

테스트는 종종 테스트 케이스 정의로 시작되며, 이는 까다로운 과정일 수 있습니다. 이 경우에는 처리하고자 하는 예시 이메일 세트와 테스트할 몇 가지 항목을 정의할 것입니다. `eval/email_dataset.py`에서 테스트 케이스를 확인할 수 있으며, 다음을 포함합니다:

1. **입력 이메일**: 다양한 이메일 예시 모음
2. **기준 분류**: `응답`, `알림`, `무시`
3. **예상 도구 호출**: 응답이 필요한 각 이메일에 대해 호출되는 도구
4. **응답 기준**: 회신이 필요한 이메일에 대한 적절한 응답 조건

다음과 같은 테스트 유형이 포함됨을 유의하십시오:
- 종단 간 "통합" 테스트 (예: 입력 이메일 -> 에이전트 -> 최종 출력 vs 응답 기준)
- 워크플로우 특정 단계 테스트 (예: 입력 이메일 -> 에이전트 -> 분류 vs 기준 분류)